## Import

In [ ]:
import sys, os
path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir)))
print(path2add)

if (not (path2add in sys.path)) :
    sys.path.append(path2add)

In [ ]:
from scipy.stats import mode
from tqdm.notebook import tqdm
from pathlib import Path

import beamtest_analysis_helper as helper
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use('CMS')

## Specify board information

In [ ]:
chip_labels = [0, 1, 2, 3]

## DESY TB Dec2023, Run39 configs
# chip_names = ["EPIR_Pair1", "BAR_4", "BAR_6", "BB_1-3"]
# high_voltages = [260, 260, 260, 200]
# offsets = [15, 10, 15, 10]
# chip_fignames = chip_names
# chip_figtitles = [
#     f"EPIR Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
#     f"Barcelona 4 HV{high_voltages[1]}V OS:{offsets[1]}",
#     f"Barcelona 6 HV{high_voltages[2]}V OS:{offsets[2]}",
#     f"BB 1-3 HV{high_voltages[3]}V OS:{offsets[3]}"
# ]

## DESY TB Feb2024, Run36 configs
# chip_names = ["EPIR_Pair1", "BAR_4", "BAR_5", "EPIR_Pair4"]
# high_voltages = [250, 260, 210, 200]
# offsets = [15, 20, 20, 15]
# chip_fignames = chip_names
# chip_figtitles = [
#     f"EPIR Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
#     f"Barcelona 4 HV{high_voltages[1]}V OS:{offsets[1]}",
#     f"Barcelona 5 HV{high_voltages[2]}V OS:{offsets[2]}",
#     f"EPIR Pair4 HV{high_voltages[3]}V OS:{offsets[3]}"
# ]

## DESY TB Apr2024, Run13 configs
chip_names = ["BAR_4", "BAR_12", "BAR_13", "BB_1-3"]
high_voltages = [260, 120, 120, 200]
offsets = [20, 20, 20, 20]
chip_fignames = chip_names
chip_figtitles = [
    f"Barcelona 4 HV{high_voltages[0]}V OS:{offsets[0]}",
    f"Barcelona 12 HV{high_voltages[1]}V OS:{offsets[1]}",
    f"Barcelona 13 HV{high_voltages[2]}V OS:{offsets[2]}",
    f"BB 1-3 HV{high_voltages[3]}V OS:{offsets[3]}"
]

## Set output directory depending on TB campaign

In [ ]:
current_dir = Path('./')
output_mother_dir = current_dir / 'etroc_TB_figs'
output_mother_dir.mkdir(exist_ok=True)

### Now you need change the directory name per campaign
### Naming rule is this:
### <TB location>_TB_MonthYear
### E.g. desy_TB_Apr2024, cern_TB_Sep2023, fnal_TB_Jul2024

output_campaign_dir = output_mother_dir / 'desy_TB_Apr2024'
output_campaign_dir.mkdir(exist_ok=True)

## Read input files

In [ ]:
files = Path('./desy_apr2024_run13').glob('loop*feather')
files = sorted(list(files))
files

In [ ]:
dataframes = []
columns_to_read = ['evt', 'ea', 'board', 'row', 'col', 'toa', 'tot', 'cal']

for idx, ifile in enumerate(files):
    tmp_df = pd.read_feather(ifile, columns=columns_to_read)
    nevt = tmp_df['evt'].nunique()
    if idx > 0:
        tmp_df['evt'] += nevt
    dataframes.append(tmp_df)
    del tmp_df

df = pd.concat(dataframes)
del dataframes

## Make occupancy map

In [ ]:
helper.plot_occupany_map(df, chipLabels=chip_labels, board_names=chip_names, tb_loc='desy', fname_tag='inclusive', save_mother_dir=output_campaign_dir)

In [ ]:
# h_inclusive = helper.return_hist_pivot(input_df=df, chipNames=chip_names, board_id_to_analyze=chip_labels, hist_bins=[100, 128, 128])
# for iboard in chip_labels:
#     helper.plot_1d_TDC_histograms(h_inclusive, chip_names[iboard], 'desy', chip_figtitles[iboard], slide_friendly=False, save_mother_dir=output_campaign_dir, tag='inclusive')
# del h_inclusive

## CAL code filtering

In [ ]:
df['identifier'] = df.groupby(['evt', 'board']).cumcount()
df['identifier'] = df['identifier'].astype(np.uint8)
cal_table = df.pivot_table(index=["row", "col"], columns=["board"], values=["cal"], aggfunc=lambda x: x.mode().iat[0])

In [ ]:
cal_table.reset_index(inplace=True)
cal_table = cal_table.set_index([('row', ''), ('col', '')]).stack().reset_index()
cal_table.columns = ['row', 'col', 'board', 'cal_mode']

In [ ]:
merged_df = pd.merge(df, cal_table, on=['board', 'row', 'col'])
del df, cal_table
cal_condition = abs(merged_df['cal'] - merged_df['cal_mode']) <= 3
merged_df = merged_df[cal_condition]
merged_df = merged_df.drop(columns=['cal_mode'])
merged_df = merged_df.sort_values(['evt', 'board', 'identifier'])
cal_filtered_df = merged_df.reset_index(drop=True)
cal_filtered_df['board'] = cal_filtered_df['board'].astype(np.uint8)
del merged_df, cal_condition

In [ ]:
# h_inclusive = helper.return_hist(input_df=cal_filtered_df, chipNames=chip_names, chipLabels=[0,1,2,3], hist_bins=[100,128,128])

# for iboard in [0,1,2,3]:
#     helper.plot_1d_TDC_histograms(h_inclusive, chip_names[iboard], chip_fignames[iboard], chip_figtitles[iboard], save=False,
#                                 tag="CAL_cleaned", fig_tag=", CAL cleaned", slide_friendly=True)

# del h_inclusive

## Offline trigger selection: single hit on the trigger board & TOT requirement

In [ ]:
identifier_counts = cal_filtered_df.loc[cal_filtered_df['board'] == 0].groupby('evt')['identifier'].size()
single_trig_hit = (identifier_counts == 1)
single_trig_hit_evt = identifier_counts[single_trig_hit].index

# Filter the DataFrame for single trigger hit events
single_trig_hit_df = cal_filtered_df.loc[cal_filtered_df['evt'].isin(single_trig_hit_evt)]

# Further filter based on 'board' and 'tot' criteria
selected_evt = single_trig_hit_df.loc[(single_trig_hit_df['board'] == 0) & (single_trig_hit_df['tot'].between(100, 200))]['evt'].unique()

trig_cleaned_df = single_trig_hit_df.loc[single_trig_hit_df['evt'].isin(selected_evt)].reset_index(drop=True)

del cal_filtered_df, identifier_counts, selected_evt, single_trig_hit_df, single_trig_hit, single_trig_hit_evt

In [ ]:
# h_cleaned = helper.return_hist(input_df=trig_cleaned_df, chipNames=chip_names, chipLabels=[0,1,2,3], hist_bins=[100,128,128])
# for iboard in [0,1,2,3]:
#     helper.plot_1d_TDC_histograms(h_cleaned, chip_names[iboard], chip_fignames[iboard], chip_figtitles[iboard], save=False,
#                                 tag="offline trigger selection", fig_tag=", offline trigger selection", slide_friendly=True)
# del h_cleaned

In [ ]:
# helper.plot_number_of_hits_per_event(input_df=trig_cleaned_df, fig_titles=chip_figtitles, fig_tag='offline trigger selection', bins=15, hist_range=(0,15), do_logy=True)

In [ ]:
# helper.plot_number_of_fired_board(input_df=trig_cleaned_df, fig_tag='offline trigger selection')

## Make spatial correlation plots

In [ ]:
id1 = 0
id2 = 1

In [ ]:
single_hit_df, multi_hits_df = helper.return_broadcast_dataframe(input_df=trig_cleaned_df, reference_board_id=id2, board_id_want_broadcast=id1)

In [ ]:
helper.plot_correlation_of_pixels(single_hit_df, sorted(set(int(col.split('_')[-1]) for col in single_hit_df.columns)), board_name1=chip_names[id1], board_name2=chip_names[id2],
                                  tb_loc='desy', fname_tag='single', save_mother_dir=output_campaign_dir)

In [ ]:
helper.plot_correlation_of_pixels(multi_hits_df, sorted(set(int(col.split('_')[-1]) for col in multi_hits_df.columns)), board_name1=chip_names[id1], board_name2=chip_names[id2],
                                  tb_loc='desy', fname_tag='multiple', save_mother_dir=output_campaign_dir)

In [ ]:
helper.plot_difference_of_pixels(input_df=single_hit_df, board_ids=sorted(set(int(col.split('_')[-1]) for col in single_hit_df.columns)),
                                 tb_loc='desy', fname_tag='single', board_name1=chip_names[id1], board_name2=chip_names[id2], save_mother_dir=output_campaign_dir)

In [ ]:
helper.plot_difference_of_pixels(input_df=multi_hits_df, board_ids=sorted(set(int(col.split('_')[-1]) for col in multi_hits_df.columns)),
                                 tb_loc='desy', fname_tag='multiple', board_name1=chip_names[id1], board_name2=chip_names[id2], save_mother_dir=output_campaign_dir)